In [1]:
import os
from requests import Session

In [2]:
BASE_API_URL = "https://numerbay.ai/backend-api/v1"

NUMERBAY_USERNAME = "YOUR_USERNAME"
NUMERBAY_PASSWORD = "YOUR_PASSWORD"

PRODUCT_ID = 79  # change to your product ID
PRODUCT_ARTIFACT_FILE = "/app/notebooks/research/numerai/example_predictions.csv"  # change to your file path

In [3]:
client = Session()

# Login to get authentication token

In [4]:
login_data = {
    "username": NUMERBAY_USERNAME,
    "password": NUMERBAY_PASSWORD
}

r = client.post(f"{BASE_API_URL}/login/access-token", data=login_data)

In [5]:
access_token = r.json()['access_token']

In [6]:
# Add the token header to the client for all subsequent queries

client.headers = {"Authorization": f"Bearer {access_token}"}

# List product artifacts

In [7]:
r = client.get(
    f"{BASE_API_URL}/products/{PRODUCT_ID}/artifacts"
)
content = r.json()

In [8]:
content

{'total': 0, 'data': []}

# Generate upload URL for product artifact

In [9]:
artifact_data = {
    "filename": os.path.basename(PRODUCT_ARTIFACT_FILE)  # for convenience we don't add the optional file size info here
}

r = client.post(
    f"{BASE_API_URL}/products/{PRODUCT_ID}/artifacts/generate-upload-url",
    data=artifact_data,
)

content = r.json()
artifact_id = content['id']
upload_url = content['url']

# Upload the file to the generated URL

In [10]:
r = client.put(upload_url, data=open(PRODUCT_ARTIFACT_FILE, 'rb'), headers={"Content-type": "application/octet-stream"})  # The header override is important

# Validate the upload
If the upload was a success by calling the API below, you will receive an artifact upload success email. Confirmed buyers will receive their notifications and submissions as well.

In [11]:
r = client.post(
    f"{BASE_API_URL}/products/{PRODUCT_ID}/artifacts/{artifact_id}/validate-upload",
)

In [12]:
r.status_code  # this would be 404 if the upload failed

200

# List product artifacts

In [13]:
r = client.get(
    f"{BASE_API_URL}/products/{PRODUCT_ID}/artifacts",
)
content = r.json()

In [14]:
content

{'total': 1,
 'data': [{'date': '2021-10-13T16:01:38.805287',
   'round_tournament': 286,
   'description': None,
   'url': None,
   'object_name': 'numerai-predictions-numerbay_286_f44d046b650642b4b9476ddfb04b2d65.csv',
   'object_size': None,
   'state': 'active',
   'id': 123,
   'product_id': 79}]}